In [1]:
import numpy as np


In [2]:
nrows = 4
ncols = 4

In [11]:
position = np.array()

TypeError: Required argument 'object' (pos 1) not found

In [10]:
for i in range(nrows * ncols):
    if (i % nrows) == 0 and i != 0: 
        print("")
    print("[]", end='')
        

[][][][]
[][][][]
[][][][]
[][][][]